In [ ]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import librosa.display
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import pathlib
import os

# 데이터 로드 및 전처리 함수
def preprocess_test_data(data_path, num_mfcc=100, n_fft=2048, hop_length=512, max_pad_len=87):
    # 음성 데이터 불러오기
    y, sr = librosa.load(data_path)  # 오디오 데이터 로드
    
    # MFCC 추출
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
    
    # 패딩 적용
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')  # 패딩
    
    return mfccs[..., np.newaxis]  # 마지막 차원 추가

# 모델 로드
#MODEL_SAVE_PATH = 'model/2024_05_02.h5'
#model = load_model(MODEL_SAVE_PATH)

# 레이블 로드 (학습된 데이터셋의 레이블)
#DATASET_PATH = 'training_data/new_sound/FIRE/01_03_709_220915_0022_11_SN.wav'
#data_dir = pathlib.Path(DATASET_PATH)
labels = ['CARBBANG', 'CARSTOP', 'EMERGENCY', 'FASTEMERGENCY', 'FIRE', 'GAS', 'SHOUT', 'STEALSOUND']
#labels = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d)) and d != '.ipynb_checkpoints']

# LabelEncoder 초기화
le = LabelEncoder()
le.fit(labels)

# 테스트할 .wav 파일 경로
TEST_WAV_PATH = 'training_data/new_sound/NoSound/10__33 (3).wav'  # 테스트할 파일 경로로 변경

# 테스트 데이터 전처리
test_mfcc = preprocess_test_data(TEST_WAV_PATH, max_pad_len=87)

# 모델을 통해 예측
predictions = model.predict(np.array([test_mfcc]))  # 입력 데이터 차원 추가
predicted_probabilities = predictions[0]

# 가장 높은 확률의 레이블 예측
predicted_label = le.inverse_transform([np.argmax(predictions)])[0]

# 예측 결과 확률 막대그래프로 그리기
plt.figure(figsize=(10, 6))
plt.bar(labels, predicted_probabilities)
plt.title(f'Predicted probabilities for file: {TEST_WAV_PATH}')
plt.xlabel('Labels')
plt.ylabel('Probability')
plt.show()

print(f"Predicted label: {predicted_label}")
